In [28]:
import pandas as pd
import mysql.connector

In [29]:
cell_phones_df = pd.read_csv("cell_phones_total.csv")
population_df = pd.read_csv("pop.csv")




In [30]:
df_phones.columns = cell_phones_df.columns.str.strip()
df_population.columns = population_df.columns.str.strip()


In [31]:
cell_phones_df = cell_phones_df.melt(id_vars=['country'], var_name='Year', value_name='Cell_Phone_Total')
population_df = population_df.melt(id_vars=['country'], var_name='Year', value_name='Population')


In [32]:
cell_phones_df.rename(columns={'country': 'Country'}, inplace=True)
population_df.rename(columns={'country': 'Country'}, inplace=True)
cell_phones_df['Year'] = cell_phones_df['Year'].astype(int)
population_df['Year'] = population_df['Year'].astype(int)


In [33]:
db = mysql.connector.connect(host="localhost",user="root",password="Logesh",database="bootcamp")
cursor = db.cursor()
print(db)
cursor = db.cursor()


In [34]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS CellPhones (
        Country VARCHAR(100),
        Year INT,
        Cell_Phone_Total BIGINT,
        PRIMARY KEY (Country, Year)
    )
""")


In [39]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Population (
        Country VARCHAR(100),
        Year INT,
        Population VARCHAR(20),  # VARCHAR to handle values like "3.28M"
        PRIMARY KEY (Country, Year)
    )
""")


In [40]:
db.commit()
print("Tables created successfully!")


Tables created successfully!


In [41]:
for _, row in cell_phones_df.iterrows():
    cell_phone_total = row['Cell_Phone_Total'] if pd.notna(row['Cell_Phone_Total']) else 0
    cursor.execute(
        "INSERT IGNORE INTO CellPhones (Country, Year, Cell_Phone_Total) VALUES (%s, %s, %s)",
        (row['Country'], row['Year'], cell_phone_total)
    )

for _, row in population_df.iterrows():
    cursor.execute(
        "INSERT IGNORE INTO Population (Country, Year, Population) VALUES (%s, %s, %s)",
        (row['Country'], row['Year'], row['Population'])
    )





In [43]:
db.commit()
print("Data inserted successfully!")


Data inserted successfully!


In [44]:
cursor.execute("""
    UPDATE CellPhones 
    SET Cell_Phone_Total = 0 
    WHERE Cell_Phone_Total IS NULL;
""")


In [45]:
db.commit()
print("Updated NULL values to 0.")



Updated NULL values to 0.


In [46]:
cursor.execute("SELECT MAX(Year) FROM CellPhones;")
latest_year = cursor.fetchone()[0]
print(f"Latest Year: {latest_year}")


Latest Year: 2023


In [47]:
cursor.execute(f"""
    SELECT Country, Cell_Phone_Total 
    FROM CellPhones 
    WHERE Year = {latest_year}
    ORDER BY Cell_Phone_Total DESC
    LIMIT 10;
""")
top_countries = cursor.fetchall()

print("\n🔝 Top 10 Countries by Mobile Phone Usage in", latest_year)
for country in top_countries:
    print(country)



🔝 Top 10 Countries by Mobile Phone Usage in 2023
('Suriname', 989)
('Luxembourg', 961)
('Comoros', 934)
('Bhutan', 752)
('Malta', 749)
('Maldives', 745)
('Cape Verde', 590)
('Djibouti', 574)
('Brunei', 541)
('Iceland', 478)


In [48]:
cursor.execute("""
    SELECT Year, Cell_Phone_Total 
    FROM CellPhones 
    WHERE Country = 'India'
    ORDER BY Year ASC;
""")
india_growth = cursor.fetchall()

print("\n📈 Mobile Growth in India")
for year, phones in india_growth:
    print(f"{year}: {phones}")



📈 Mobile Growth in India
1960: 0
1961: 0
1962: 0
1963: 0
1964: 0
1965: 0
1966: 0
1967: 0
1968: 0
1969: 0
1970: 0
1971: 0
1972: 0
1973: 0
1974: 0
1975: 0
1976: 0
1977: 0
1978: 0
1979: 0
1980: 0
1981: 0
1982: 0
1983: 0
1984: 0
1985: 0
1986: 0
1987: 0
1988: 0
1989: 0
1990: 0
1991: 0
1992: 0
1993: 0
1994: 0
1995: 77
1996: 328
1997: 882
1998: 1
1999: 2
2000: 4
2001: 7
2002: 13
2003: 34
2004: 52
2005: 90
2006: 166
2007: 234
2008: 347
2009: 525
2010: 752
2011: 894
2012: 865
2013: 886
2014: 944
2015: 1
2016: 1
2017: 1
2018: 1
2019: 1
2020: 1
2021: 1
2022: 1
2023: 0


In [49]:
df = pd.read_sql("SELECT * FROM CellPhones WHERE Year = 2022 ORDER BY Cell_Phone_Total DESC;", db)
df.to_csv("top_mobile_usage_2022.csv", index=False)
print("Data exported successfully to top_mobile_usage_2022.csv")




Data exported successfully to top_mobile_usage_2022.csv


C:\Users\loges\AppData\Local\Temp\ipykernel_9052\2135104897.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM CellPhones WHERE Year = 2022 ORDER BY Cell_Phone_Total DESC;", db)


In [51]:
cursor.close()
db.close()
print("MySQL connection closed.")


MySQL connection closed.
